In [10]:
import requests
from datetime import datetime
import pandas as pd

In [3]:
# Code voor het activeren van de API

# URL and headers
url = "https://api.openraadsinformatie.nl/v1/elastic/ori_*/_search?"
headers = {
    "Content-Type": "application/json"
}

# Request JSON data
data = {
    "query": {
        "bool": {
            "must": [
                {
                    "simple_query_string": {
                        "fields": ["text", "title", "description", "name"],
                        "default_operator": "or",
                        "query": "*"
                    }
                },
                {
                    "terms": {
                        "_index": ["ori_hoorn_20230821161431"],
                    }
                }
            ]
        }
    },
    "size": 10000,
    "_source": {
        "includes": ["*"],
        "excludes": []
    },
    "from": 0,
    "sort": [
        {
            "_score": {
                "order": "desc"
            }
        }
    ],
}


# Post request versturen
response = requests.post(url, json=data, headers=headers)

# Check de response
if response.status_code == 200:
    result = response.json()

else:
    print("Request failed with status code:", response.status_code)
    print("Response content:", response.text)


In [4]:
print(len(result['hits']['hits']))

all_results = {r['_id']: r for r in result['hits']['hits']}

6154


In [32]:
def process_report(id, all_results):
    json_data = dict()
    
    data = all_results[id]

    if 'attachment' in data['_source']:

        json_data['id'] = id
        json_data['dc_title'] = data['_source']['name']   
        json_data['dc_source'] = f"https://hoorn.bestuurlijkeinformatie.nl/Reports/Item/{data['_source']['was_generated_by']['original_identifier']}"
        json_data['dc_type'] = data['_source']['@type']   
        json_data['foi_classificaties'] = data['_source']['classification']
        json_data['dc_publisher'] = all_results[data['_source']['has_organization_name']]['_source']['name']
        try:
            json_data['foi_publishedDate'] = datetime.fromisoformat(data['_source']['start_date']).strftime('%Y-%m-%d')

        except KeyError:
            json_data['foi_publishedDate'] = datetime.fromisoformat(data['_source']['was_generated_by']['started_at_time']).strftime('%Y-%m-%d')

        if 'attachment' in data['_source']:

            attachments = data['_source']['attachment']
            if isinstance(attachments, str):
                attachments = [attachments]
                
            foi_files = []
        
            for a in attachments:
                try:
                    data = all_results[a]
                    temp_data = dict()
                    temp_data['dc_title'] = data['_source']['name']
                    temp_data['dc_type'] = data['_source']['@type']
                    temp_data['dc_source'] = data['_source']['url']
                    temp_data['dc_format'] = data['_source']['content_type']
                    foi_files.append(temp_data)
                except KeyError:
                    pass
            
        if foi_files == []:
            return {}
            
        json_data['foi_files'] = foi_files

    return json_data

process_report('1810297', all_results)

{'id': '1810297',
 'dc_title': 'Locatiekeuzeonderzoek skatepark Julianapark',
 'dc_source': 'https://hoorn.bestuurlijkeinformatie.nl/Reports/Item/a5154c8d-d71b-4a58-9d26-f15c27b23b00',
 'dc_type': 'Report',
 'foi_classificaties': 'Financiële documenten',
 'dc_publisher': 'Gemeente Hoorn',
 'foi_publishedDate': '2023-06-02',
 'foi_files': [{'dc_title': 'Bijlage 1 Locatiekeuzeonderzoek Skatebaan (2044765)',
   'dc_type': 'MediaObject',
   'dc_source': 'https://api.openraadsinformatie.nl/v1/resolve/ibabs/report/c318f908-e156-48ab-88ce-563eb3b13928',
   'dc_format': 'application/pdf'},
  {'dc_title': 'Bijlage 2 Participatieverslag Hoorn Skatepark (2044765)',
   'dc_type': 'MediaObject',
   'dc_source': 'https://api.openraadsinformatie.nl/v1/resolve/ibabs/report/0c8520a4-0429-4629-a563-4a2f0ba9daea',
   'dc_format': 'application/pdf'},
  {'dc_title': 'Bijlage 3 Stakeholdergesprek HVV Hollandia + Watersportvereniging Hoorn (2044765)',
   'dc_type': 'MediaObject',
   'dc_source': 'https://api

In [33]:
# En zo doe ik het voor alle results van het type Report
all_result_list = []

for k,v in all_results.items():
    if v['_source']['@type'] == 'Report':
        x = process_report(k, all_results)
        if x != {}:
            all_result_list.append(x)

In [34]:
df = pd.DataFrame(all_result_list)

df['foi_files_length'] = df['foi_files'].apply(len)

display(df)

,id,dc_title,dc_source,dc_type,foi_classificaties,dc_publisher,foi_publishedDate,foi_files,foi_files_length
0,2226834,Inkoopprocedure en Flyer aanbesteden Noord Hol...,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Financiële documenten,Gemeente Hoorn,2023-09-25,"[{'dc_title': 'Inkoopprocedure', 'dc_type': 'M...",2
1,1810306,Beplantingsplan - Boelisstraat - Van Beijerens...,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Financiële documenten,Gemeente Hoorn,2023-05-22,[{'dc_title': 'Beplantingsplan - Boelisstraat ...,1
2,1810291,Onderzoeksrapport Slavernijverleden van Hoorn,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Financiële documenten,Gemeente Hoorn,2023-06-29,[{'dc_title': 'Onderzoeksrapport Slavernijverl...,1
3,1810314,Overzicht basisscholen,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Financiële documenten,Gemeente Hoorn,2023-03-02,"[{'dc_title': '2023-Naar-de-basisschool', 'dc_...",1
4,1810308,Totaalontwerp Venenlaankwartier,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Financiële documenten,Gemeente Hoorn,2023-05-15,[{'dc_title': 'Totaalontwerp Venenlaankwartier...,1
...,...,...,...,...,...,...,...,...,...
2236,1815175,Werkzaamheden persriool langs Willemsweg,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Artikel 36 vragen,Gemeente Hoorn,2020-10-13,[{'dc_title': 'HoornLokaal - Art 36 vragen wer...,2
2237,1810561,ICT inkoopbeleid,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Moties,Gemeente Hoorn,2022-12-13,[{'dc_title': 'Motie 5 VVD D66 CU ICT Inkoopbe...,1
2238,1811263,Motie 5- Starterslening Kadernota 2018,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Moties,Gemeente Hoorn,2018-07-10,"[{'dc_title': 'Motie 5 - VVD, D66, CU, FT, SoH...",1
2239,1811266,Motie 2 -Parkeren Westfries Ijsbaan,https://hoorn.bestuurlijkeinformatie.nl/Report...,Report,Moties,Gemeente Hoorn,2018-07-10,[{'dc_title': 'Motie 2 - CDA -parkeren Westfri...,1


In [35]:
df.foi_classificaties.value_counts()

foi_classificaties
Moties                              644
RSS                                 449
Artikel 36 vragen                   347
Ingekomen stukken - College         326
Besluitenlijst B&W                  308
Toezeggingen                        127
Financiële documenten                35
Verordeningen Raad en Commissies      5
Name: count, dtype: int64

In [36]:
df.foi_files_length.value_counts()

foi_files_length
1    1484
2     590
3     118
4      32
5       9
6       4
8       2
9       1
7       1
Name: count, dtype: int64

In [37]:
df.groupby('foi_classificaties')['foi_files_length'].mean()

foi_classificaties
Artikel 36 vragen                   2.276657
Besluitenlijst B&W                  1.000000
Financiële documenten               1.400000
Ingekomen stukken - College         1.377301
Moties                              1.071429
RSS                                 1.757238
Toezeggingen                        1.314961
Verordeningen Raad en Commissies    1.000000
Name: foi_files_length, dtype: float64